# 12 layers, pre-trained weight

In [1]:
import tensorflow as tf
import numpy as np
import time
import os
import re

from PIL import Image as PI

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

In [2]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess_ :
    saver = tf.train.import_meta_graph('D:/Backup_data/jongkeun/paper/190524/save_dir-494.meta')
    saver.restore(sess_, tf.train.latest_checkpoint('D:/Backup_data/jongkeun/paper/190524/'))
    
    graph = tf.get_default_graph()
    
#     print(graph.get_tensor_by_name('layer0_conv2d_1/kernel:0'))
    weight0_1, weight0_2 = sess_.run([graph.get_tensor_by_name('layer0_conv2d_1/kernel:0'), 
                                      graph.get_tensor_by_name('layer0_conv2d_2/kernel:0')])
    weight1, weight1_1, weight1_2 = sess_.run([graph.get_tensor_by_name('layer1_conv2d/kernel:0'), 
                                               graph.get_tensor_by_name('layer1_conv2d_1/kernel:0'),
                                               graph.get_tensor_by_name('layer1_conv2d_2/kernel:0')])
    weight2, weight2_1, weight2_2 = sess_.run([graph.get_tensor_by_name('layer2_conv2d/kernel:0'), 
                                               graph.get_tensor_by_name('layer2_conv2d_1/kernel:0'),
                                               graph.get_tensor_by_name('layer2_conv2d_2/kernel:0')])
    weight3, weight3_1, weight3_2 = sess_.run([graph.get_tensor_by_name('layer3_conv2d/kernel:0'), 
                                               graph.get_tensor_by_name('layer3_conv2d_1/kernel:0'),
                                               graph.get_tensor_by_name('layer3_conv2d_2/kernel:0')])
    
    
#     weight_ = weight[:,:,:,:]
#     print(weight_)

print("++++++++++++++++++ DONE ++++++++++++++++++")

W1107 00:54:16.435084 15272 deprecation.py:323] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


++++++++++++++++++ DONE ++++++++++++++++++


In [3]:
training_epochs = 500
batch_size = 20
image_resize = 128
image_width = image_resize
image_height = image_resize
initial_rate = 1e-5
label_len = 3

home_dir = "D:/Backup_data/jongkeun"

image_file = "/dir_images/190819_newTraining/"
test_image_file = "/dir_images/190309_new_test/"

label_file = "/csv/190819_newTraining.csv"
test_label_file = "/csv/190309_new_test.csv"

In [4]:
label_dir = home_dir + label_file
test_label_dir = home_dir + test_label_file

# label_dir = os.getcwd() + "/csv/181126.csv"
# test_label_dir = os.getcwd() + "/csv/181126_test.csv"

f_label = open(label_dir, 'r')
f_test_label = open(test_label_dir, 'r')

label_data = f_label.readlines()
test_label_data = f_test_label.readlines()

f_label.close()
f_test_label.close()

label_array = np.array(np.reshape(label_data, [-1, label_len]), dtype=np.int32)
test_label_array = np.array(np.reshape(test_label_data, [-1, label_len]), dtype=np.int32)

print(len(label_array))
print(len(test_label_array))

9182
1018


In [5]:
image_dir = home_dir + image_file
test_image_dir = home_dir + test_image_file

# image_dir = os.getcwd() + "/dir_images/181126/"
# test_image_dir = os.getcwd() + "/dir_images/181126_test/"

image_list = os.listdir(image_dir)
test_image_list = os.listdir(test_image_dir)

image_list = sorted(image_list, key=lambda x: (int(re.sub('\D','',x)),x))
test_image_list = sorted(test_image_list, key=lambda y: (int(re.sub('\D','',y)),y))

image_array = []
test_image_array = []

for i in range(len(image_list)) :
    image_list[i] = image_dir + image_list[i]
    image_array.append(np.array(PI.open(image_list[i]).resize((image_resize, image_resize)).convert('L')))

for j in range(len(test_image_list)) :
    test_image_list[j] = test_image_dir + test_image_list[j]
    test_image_array.append(np.array(PI.open(test_image_list[j]).resize((image_resize, image_resize)).convert('L')))

image_array = np.reshape(image_array, [-1, image_width*image_height])
test_image_array = np.reshape(test_image_array, [-1, image_width*image_height])

image_array = np.array(np.reshape(image_array, [-1, image_width, image_height, 1]), dtype=np.float32)
test_image_array = np.array(np.reshape(test_image_array, [-1, image_width, image_height, 1]), dtype=np.float32)

print(len(image_array))
print(len(test_image_array))

9182
1018


In [6]:
x = tf.placeholder(dtype=tf.float32, shape=[None, image_width, image_height, 1], name='input_image')
y_ = tf.placeholder(dtype=tf.float32, shape=[None, label_len], name='input_label')
learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')

# W_pre_trained = tf.placeholder(dtype=tf.float32, shape=[3,3,8,128], name='pre_trained_weight')

is_training = tf.placeholder(dtype=tf.bool)

In [7]:
W_h0_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,1,8], name='W_h0_1')
W_h0_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,8,128], name='W_h0_2')

W_h1 = tf.placeholder(dtype=tf.float32, shape=[3,3,128,256], name='W_h1')
W_h1_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,256, 128], name='W_h1_1')
W_h1_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,128,256], name='W_h1_2')

W_h2 = tf.placeholder(dtype=tf.float32, shape=[3,3,256,512], name='W_h2')
W_h2_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,512,256], name='W_h2_1')
W_h2_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,256,512], name='W_h2_2')

W_h3 = tf.placeholder(dtype=tf.float32, shape=[3,3,512,512], name='W_h3')
W_h3_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,512,1024], name='W_h3_1')
W_h3_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,1024,512], name='W_h3_2')

In [8]:
def pooling(input_, dropout, is_training) :
#     layer = tf.layers.conv2d(input_data, size, [3, 3], padding='SAME', name=name_conv2d)
    pooling = tf.layers.average_pooling2d(input_, [2, 2], [2, 2], padding='SAME')
    pooling = tf.layers.dropout(pooling, dropout, is_training)
    
    return pooling

def learning_layer(input_data, size, dropout, is_training) :
    layer = tf.layers.conv2d(input_data, size, [3, 3], padding='SAME')
    layer = tf.layers.average_pooling2d(layer, [2, 2], [2, 2], padding='SAME')
    layer = tf.layers.dropout(layer, dropout, is_training)
    
    return layer

def convolutional(input_, weight_) :
    conv_ = tf.nn.conv2d(input_, weight_, strides=[1,1,1,1], padding='SAME')
    hidden_ = tf.nn.relu(conv_)
    
    return hidden_

with tf.device('/gpu:0') :
    layer0_1 = convolutional(x, W_h0_1)
    layer0_2 = convolutional(layer0_1, W_h0_2)
    layer1 = convolutional(layer0_2, W_h1)
    layer1_ = pooling(layer1, dropout=0.7, is_training=is_training)

    layer1_1 = convolutional(layer1_, W_h1_1)
    layer1_2 = convolutional(layer1_1, W_h1_2)
    layer2 = convolutional(layer1_2, W_h2)
    layer2_ = pooling(layer2, dropout=0.7, is_training=is_training)

    layer2_1 = convolutional(layer2_, W_h2_1)
    layer2_2 = convolutional(layer2_1, W_h2_2)
    layer3 = convolutional(layer2_2, W_h3)
    layer3_ = pooling(layer3, dropout=0.7, is_training=is_training)

    layer3_1 = convolutional(layer3_, W_h3_1)
    layer3_2 = convolutional(layer3_1, W_h3_2)
    layer4_ = learning_layer(input_data=layer3_2, size=1024, dropout=0.7, is_training=is_training)

W1107 00:57:13.324799 15272 deprecation.py:323] From <ipython-input-8-6ce99a79309f>:3: average_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.AveragePooling2D instead.
W1107 00:57:13.431514 15272 deprecation.py:323] From <ipython-input-8-6ce99a79309f>:4: dropout (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dropout instead.
W1107 00:57:13.492324 15272 deprecation.py:323] From <ipython-input-8-6ce99a79309f>:9: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W1107 00:57:13.495315 15272 deprecation.py:506] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is d

In [9]:
fully_layer = tf.contrib.layers.flatten(layer4_)
fully_layer = tf.layers.dense(fully_layer, 1024, activation=tf.nn.relu)
fully_layer = tf.layers.dropout(fully_layer, 0.5, is_training)

fully_layer2 = tf.layers.dense(fully_layer, 512, activation=tf.nn.relu)
fully_layer2 = tf.layers.dropout(fully_layer2, 0.5, is_training)

#     fully_layer3 = tf.layers.dense(fully_layer2, 256, activation=tf.nn.relu, name='fully_connected_layer3')
#     fully_layer3 = tf.layers.dropout(fully_layer3, 0.5, is_training, name='fully_connected_dropout3')

W1107 00:57:15.741555 15272 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1107 00:57:15.742537 15272 deprecation.py:323] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\contrib\layers\python\layers\layers.py:1634: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W1107 00:57:15.920090 15272 deprecation.py:323] From <ipython-input-9-55154374ecaa>:2: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


In [10]:
logits = tf.layers.dense(fully_layer2, label_len, activation=None)

with tf.name_scope('pred') : 
    pred = tf.nn.softmax(logits)
with tf.name_scope('loss') :
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_))

with tf.name_scope('train') :
        train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

label_output = tf.argmax(pred, 1)
label_input = tf.argmax(y_, 1)

In [11]:
config = tf.ConfigProto(log_device_placement = True, allow_soft_placement=True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess :
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess, coord=coord)
        
    sess.run(tf.global_variables_initializer())
    
    total_batch = int(len(image_list) / batch_size)
    correct_list = []
    incorrect_list = []
    cost_list = []
    cost_sum_list = []
    pred_list = []
    total_acc = 0
    cost_sum = 0
    cost_flag = 0
    rate_ = 1
    
    start_time = time.time()
    
    print("")
    
    for epoch in range(training_epochs) :
        total_cost = 0
        
        if epoch%50 == 49 :
                rate_ *= 0.8
                
        if epoch%10 == 9 :
#             rate_ *= 0.8
            midpoint = int(time.time() - start_time)
        
            print("")
            print("=======================================================================================")
            print('{:03d}:{:02d}:{:02d}'.format(midpoint//3600, (midpoint%3600//60), midpoint%60))
            print("=======================================================================================")
            print("")
        
        batch_index = np.random.choice(len(image_array), total_batch, replace=False)
                
        for i in range(total_batch) :                
            _, _loss = sess.run([train, loss], feed_dict={x: image_array[[batch_index[i]]], y_: label_array[[batch_index[i]]], 
                                                          is_training: True, learning_rate: initial_rate*rate_,
                                                          W_h0_1: weight0_1, W_h0_2: weight0_2,
                                                          W_h1: weight1, W_h1_1: weight1_1, W_h1_2: weight1_2,
                                                          W_h2: weight2, W_h2_1: weight2_1, W_h2_2: weight2_2,
                                                          W_h3: weight3, W_h3_1: weight3_1, W_h3_2: weight3_2})                 
            total_cost += _loss

        avg_cost = total_cost/total_batch
        print('Epoch : ', '%4d' % (epoch + 1), '    Avg. cost = ', '{:.4f}'.format(avg_cost))            
        
#         train_writer.add_summary(_merge, epoch)
               
    print("")
    print("=======================================================================================")
    print("================================     Training done     ================================")
    print("=======================================================================================")
    print("")
    
    for test in range(len(test_image_array)) :
        _acc, _label_input, _label_output, _pred = sess.run([acc, label_input, label_output, pred], 
                             feed_dict={x: test_image_array[[test]], y_: test_label_array[[test]], is_training: False,
                                        W_h0_1: weight0_1, W_h0_2: weight0_2,
                                        W_h1: weight1, W_h1_1: weight1_1, W_h1_2: weight1_2,
                                        W_h2: weight2, W_h2_1: weight2_1, W_h2_2: weight2_2,
                                        W_h3: weight3, W_h3_1: weight3_1, W_h3_2: weight3_2})

        total_acc += _acc
        pred_list.append(list(_pred[0]))
        
        print("n: ", test, "    label_input: ", _label_input, "    label_output: ", _label_output)
        
        if _label_input == _label_output :
            correct_list.append(test)   
        else :
            incorrect_list.append(test)
        
    print("")
    print("          TOTAL ACC    : ", '{:.5f}'.format(total_acc / len(test_image_array)))
    print("")
    print(pred_list)
    
    end_time = int(time.time() - start_time)
    
    print("")
    print("=======================================================================================")
    print('{:03d}:{:02d}:{:02d}'.format(end_time//3600, (end_time%3600//60), end_time%60))
    print("=======================================================================================")
    print("")
    
    coord.request_stop()
    coord.join(thread)

W1107 00:57:16.546398 15272 deprecation.py:323] From <ipython-input-11-4cb70f9ea11e>:6: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W1107 00:57:16.547385 15272 queue_runner_impl.py:471] `tf.train.start_queue_runners()` was called when no queue runners were defined. You can safely remove the call to this deprecated function.



Epoch :     1     Avg. cost =  0.7966
Epoch :     2     Avg. cost =  0.6176
Epoch :     3     Avg. cost =  0.6289
Epoch :     4     Avg. cost =  0.6150
Epoch :     5     Avg. cost =  0.5953
Epoch :     6     Avg. cost =  0.6498
Epoch :     7     Avg. cost =  0.5301
Epoch :     8     Avg. cost =  0.5604
Epoch :     9     Avg. cost =  0.5965

000:05:11

Epoch :    10     Avg. cost =  0.5601
Epoch :    11     Avg. cost =  0.5470
Epoch :    12     Avg. cost =  0.5243
Epoch :    13     Avg. cost =  0.5170
Epoch :    14     Avg. cost =  0.5339
Epoch :    15     Avg. cost =  0.5199
Epoch :    16     Avg. cost =  0.5110
Epoch :    17     Avg. cost =  0.5167
Epoch :    18     Avg. cost =  0.5034
Epoch :    19     Avg. cost =  0.4437

000:10:53

Epoch :    20     Avg. cost =  0.4998
Epoch :    21     Avg. cost =  0.5244
Epoch :    22     Avg. cost =  0.4924
Epoch :    23     Avg. cost =  0.5325
Epoch :    24     Avg. cost =  0.4942
Epoch :    25     Avg. cost =  0.4774
Epoch :    26     Avg. co

Epoch :   148     Avg. cost =  0.2961
Epoch :   149     Avg. cost =  0.2407

001:25:21

Epoch :   150     Avg. cost =  0.2485
Epoch :   151     Avg. cost =  0.2248
Epoch :   152     Avg. cost =  0.2242
Epoch :   153     Avg. cost =  0.2681
Epoch :   154     Avg. cost =  0.2323
Epoch :   155     Avg. cost =  0.2383
Epoch :   156     Avg. cost =  0.2420
Epoch :   157     Avg. cost =  0.2511
Epoch :   158     Avg. cost =  0.2385
Epoch :   159     Avg. cost =  0.2301

001:31:03

Epoch :   160     Avg. cost =  0.2536
Epoch :   161     Avg. cost =  0.2654
Epoch :   162     Avg. cost =  0.2087
Epoch :   163     Avg. cost =  0.2126
Epoch :   164     Avg. cost =  0.2256
Epoch :   165     Avg. cost =  0.2158
Epoch :   166     Avg. cost =  0.2175
Epoch :   167     Avg. cost =  0.2178
Epoch :   168     Avg. cost =  0.2579
Epoch :   169     Avg. cost =  0.2149

001:36:46

Epoch :   170     Avg. cost =  0.2530
Epoch :   171     Avg. cost =  0.2217
Epoch :   172     Avg. cost =  0.2195
Epoch :   173 

Epoch :   290     Avg. cost =  0.0920
Epoch :   291     Avg. cost =  0.1443
Epoch :   292     Avg. cost =  0.1019
Epoch :   293     Avg. cost =  0.0961
Epoch :   294     Avg. cost =  0.1246
Epoch :   295     Avg. cost =  0.0987
Epoch :   296     Avg. cost =  0.1212
Epoch :   297     Avg. cost =  0.0971
Epoch :   298     Avg. cost =  0.0963
Epoch :   299     Avg. cost =  0.1087

002:51:02

Epoch :   300     Avg. cost =  0.1059
Epoch :   301     Avg. cost =  0.1134
Epoch :   302     Avg. cost =  0.1102
Epoch :   303     Avg. cost =  0.0963
Epoch :   304     Avg. cost =  0.0914
Epoch :   305     Avg. cost =  0.0842
Epoch :   306     Avg. cost =  0.1152
Epoch :   307     Avg. cost =  0.0805
Epoch :   308     Avg. cost =  0.0837
Epoch :   309     Avg. cost =  0.0957

002:56:44

Epoch :   310     Avg. cost =  0.0941
Epoch :   311     Avg. cost =  0.0888
Epoch :   312     Avg. cost =  0.1093
Epoch :   313     Avg. cost =  0.0903
Epoch :   314     Avg. cost =  0.0850
Epoch :   315     Avg. cos

Epoch :   437     Avg. cost =  0.0312
Epoch :   438     Avg. cost =  0.0457
Epoch :   439     Avg. cost =  0.0483

004:10:58

Epoch :   440     Avg. cost =  0.0523
Epoch :   441     Avg. cost =  0.0442
Epoch :   442     Avg. cost =  0.0424
Epoch :   443     Avg. cost =  0.0394
Epoch :   444     Avg. cost =  0.0516
Epoch :   445     Avg. cost =  0.0488
Epoch :   446     Avg. cost =  0.0321
Epoch :   447     Avg. cost =  0.0469
Epoch :   448     Avg. cost =  0.0448
Epoch :   449     Avg. cost =  0.0390

004:16:40

Epoch :   450     Avg. cost =  0.0250
Epoch :   451     Avg. cost =  0.0383
Epoch :   452     Avg. cost =  0.0505
Epoch :   453     Avg. cost =  0.0509
Epoch :   454     Avg. cost =  0.0277
Epoch :   455     Avg. cost =  0.0304
Epoch :   456     Avg. cost =  0.0283
Epoch :   457     Avg. cost =  0.0279
Epoch :   458     Avg. cost =  0.0361
Epoch :   459     Avg. cost =  0.0307

004:22:23

Epoch :   460     Avg. cost =  0.0250
Epoch :   461     Avg. cost =  0.0448
Epoch :   462 

...

n:  237     label_input:  [0]     label_output:  [0]
n:  238     label_input:  [0]     label_output:  [0]
n:  239     label_input:  [0]     label_output:  [0]
n:  240     label_input:  [0]     label_output:  [0]
n:  241     label_input:  [0]     label_output:  [0]
n:  242     label_input:  [0]     label_output:  [0]
n:  243     label_input:  [0]     label_output:  [0]
n:  244     label_input:  [0]     label_output:  [1]
n:  245     label_input:  [0]     label_output:  [0]
n:  246     label_input:  [0]     label_output:  [0]
n:  247     label_input:  [0]     label_output:  [1]
n:  248     label_input:  [0]     label_output:  [0]
n:  249     label_input:  [0]     label_output:  [2]
n:  250     label_input:  [0]     label_output:  [0]
n:  251     label_input:  [0]     label_output:  [0]
n:  252     label_input:  [0]     label_output:  [0]
n:  253     label_input:  [0]     label_output:  [1]
n:  254     label_input:  [0]     label_output:  [0]
n:  255     label_input:  [0]     label_output

n:  396     label_input:  [1]     label_output:  [1]
n:  397     label_input:  [1]     label_output:  [1]
n:  398     label_input:  [1]     label_output:  [1]
n:  399     label_input:  [1]     label_output:  [1]
n:  400     label_input:  [1]     label_output:  [1]
n:  401     label_input:  [1]     label_output:  [1]
n:  402     label_input:  [1]     label_output:  [1]
n:  403     label_input:  [1]     label_output:  [0]
n:  404     label_input:  [1]     label_output:  [1]
n:  405     label_input:  [1]     label_output:  [1]
n:  406     label_input:  [1]     label_output:  [1]
n:  407     label_input:  [1]     label_output:  [1]
n:  408     label_input:  [1]     label_output:  [1]
n:  409     label_input:  [1]     label_output:  [1]
n:  410     label_input:  [1]     label_output:  [1]
n:  411     label_input:  [1]     label_output:  [1]
n:  412     label_input:  [1]     label_output:  [1]
n:  413     label_input:  [1]     label_output:  [0]
n:  414     label_input:  [1]     label_output

n:  551     label_input:  [1]     label_output:  [1]
n:  552     label_input:  [1]     label_output:  [1]
n:  553     label_input:  [1]     label_output:  [1]
n:  554     label_input:  [1]     label_output:  [1]
n:  555     label_input:  [1]     label_output:  [1]
n:  556     label_input:  [1]     label_output:  [2]
n:  557     label_input:  [1]     label_output:  [1]
n:  558     label_input:  [1]     label_output:  [1]
n:  559     label_input:  [1]     label_output:  [1]
n:  560     label_input:  [1]     label_output:  [2]
n:  561     label_input:  [1]     label_output:  [1]
n:  562     label_input:  [1]     label_output:  [1]
n:  563     label_input:  [1]     label_output:  [1]
n:  564     label_input:  [1]     label_output:  [1]
n:  565     label_input:  [1]     label_output:  [1]
n:  566     label_input:  [1]     label_output:  [1]
n:  567     label_input:  [1]     label_output:  [1]
n:  568     label_input:  [2]     label_output:  [2]
n:  569     label_input:  [2]     label_output

n:  710     label_input:  [2]     label_output:  [1]
n:  711     label_input:  [2]     label_output:  [2]
n:  712     label_input:  [2]     label_output:  [2]
n:  713     label_input:  [2]     label_output:  [2]
n:  714     label_input:  [2]     label_output:  [2]
n:  715     label_input:  [2]     label_output:  [2]
n:  716     label_input:  [2]     label_output:  [0]
n:  717     label_input:  [2]     label_output:  [2]
n:  718     label_input:  [2]     label_output:  [2]
n:  719     label_input:  [2]     label_output:  [2]
n:  720     label_input:  [2]     label_output:  [2]
n:  721     label_input:  [2]     label_output:  [2]
n:  722     label_input:  [2]     label_output:  [2]
n:  723     label_input:  [2]     label_output:  [2]
n:  724     label_input:  [2]     label_output:  [2]
n:  725     label_input:  [2]     label_output:  [2]
n:  726     label_input:  [2]     label_output:  [1]
n:  727     label_input:  [2]     label_output:  [2]
n:  728     label_input:  [2]     label_output

n:  869     label_input:  [2]     label_output:  [2]
n:  870     label_input:  [2]     label_output:  [2]
n:  871     label_input:  [2]     label_output:  [1]
n:  872     label_input:  [2]     label_output:  [2]
n:  873     label_input:  [2]     label_output:  [2]
n:  874     label_input:  [2]     label_output:  [2]
n:  875     label_input:  [2]     label_output:  [0]
n:  876     label_input:  [2]     label_output:  [0]
n:  877     label_input:  [2]     label_output:  [2]
n:  878     label_input:  [2]     label_output:  [0]
n:  879     label_input:  [2]     label_output:  [2]
n:  880     label_input:  [2]     label_output:  [2]
n:  881     label_input:  [2]     label_output:  [2]
n:  882     label_input:  [2]     label_output:  [2]
n:  883     label_input:  [2]     label_output:  [2]
n:  884     label_input:  [2]     label_output:  [2]
n:  885     label_input:  [2]     label_output:  [2]
n:  886     label_input:  [2]     label_output:  [2]
n:  887     label_input:  [2]     label_output